# LAB | Feature Engineering

**Load the data**

In this challenge, we will be working with the same Spaceship Titanic data, like the previous Lab. The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

In [1]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

PassengerId HomePlanet CryoSleep  Cabin  Destination   Age    VIP  \
0     0001_01     Europa     False  B/0/P  TRAPPIST-1e  39.0  False   
1     0002_01      Earth     False  F/0/S  TRAPPIST-1e  24.0  False   
2     0003_01     Europa     False  A/0/S  TRAPPIST-1e  58.0   True   
3     0003_02     Europa     False  A/0/S  TRAPPIST-1e  33.0  False   
4     0004_01      Earth     False  F/1/S  TRAPPIST-1e  16.0  False   

   RoomService  FoodCourt  ShoppingMall     Spa  VRDeck               Name  \
0          0.0        0.0           0.0     0.0     0.0    Maham Ofracculy   
1        109.0        9.0          25.0   549.0    44.0       Juanna Vines   
2         43.0     3576.0           0.0  6715.0    49.0      Altark Susent   
3          0.0     1283.0         371.0  3329.0   193.0       Solam Susent   
4        303.0       70.0         151.0   565.0     2.0  Willy Santantines   

   Transported  
0        False  
1         True  
2        False  
3        False  
4         True

**Check the shape of your data**

In [3]:
#your code here
spaceship.shape, list(spaceship.columns)

((8693, 14),
 ['PassengerId',
  'HomePlanet',
  'CryoSleep',
  'Cabin',
  'Destination',
  'Age',
  'VIP',
  'RoomService',
  'FoodCourt',
  'ShoppingMall',
  'Spa',
  'VRDeck',
  'Name',
  'Transported'])

**Check for data types**

In [4]:
#your code here
spaceship.dtypes
# (si quieres más detalle)
# spaceship.info()

PassengerId      object
HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported        bool
dtype: object

**Check for missing values**

In [5]:
#your code here
spaceship.isna().sum().sort_values(ascending=False)
# (y porcentaje)
# (spaceship.isna().mean()*100).round(2).sort_values(ascending=False)

CryoSleep       217
ShoppingMall    208
VIP             203
HomePlanet      201
Name            200
Cabin           199
VRDeck          188
Spa             183
FoodCourt       183
Destination     182
RoomService     181
Age             179
PassengerId       0
Transported       0
dtype: int64

There are multiple strategies to handle missing data

- Removing all rows or all columns containing missing data.
- Filling all missing values with a value (mean in continouos or mode in categorical for example).
- Filling all missing values with an algorithm.

For this exercise, because we have such low amount of null values, we will drop rows containing any missing value. 

In [6]:
#your code here
spaceship_clean = spaceship.dropna().reset_index(drop=True)
spaceship.shape, spaceship_clean.shape

((8693, 14), (6606, 14))

- **Cabin** is too granular - transform it in order to obtain {'A', 'B', 'C', 'D', 'E', 'F', 'G', 'T'}

In [7]:
#your code here
# Si ya hiciste dropna, no habrá NaN en Cabin; por si acaso:
spaceship_clean["Cabin"] = spaceship_clean["Cabin"].astype(str).str[0]
# (Opcional) limitar a las letras válidas
valid_decks = set(list("ABCDEFGT"))
spaceship_clean["Cabin"] = spaceship_clean["Cabin"].where(spaceship_clean["Cabin"].isin(valid_decks), "Unknown")

- Drop PassengerId and Name

In [8]:
#your code here
cols_to_drop = [c for c in ["PassengerId", "Name"] if c in spaceship_clean.columns]
spaceship_fe = spaceship_clean.drop(columns=cols_to_drop).copy()
spaceship_fe.columns, spaceship_fe.shape

(Index(['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age', 'VIP',
        'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
        'Transported'],
       dtype='object'),
 (6606, 12))

- For non-numerical columns, do dummies.

In [9]:
#your code here
# Asegura que el target quede numérico (si existe)
if "Transported" in spaceship_fe.columns and spaceship_fe["Transported"].dtype == "bool":
    spaceship_fe["Transported"] = spaceship_fe["Transported"].astype(int)

# One-hot encoding de todas las no numéricas
non_numeric = spaceship_fe.select_dtypes(exclude=["number"]).columns.tolist()
spaceship_dum = pd.get_dummies(spaceship_fe, columns=non_numeric, drop_first=True)

spaceship_dum.shape, spaceship_dum.head()

((6606, 20),
     Age  RoomService  FoodCourt  ShoppingMall     Spa  VRDeck  Transported  \
 0  39.0          0.0        0.0           0.0     0.0     0.0            0   
 1  24.0        109.0        9.0          25.0   549.0    44.0            1   
 2  58.0         43.0     3576.0           0.0  6715.0    49.0            0   
 3  33.0          0.0     1283.0         371.0  3329.0   193.0            0   
 4  16.0        303.0       70.0         151.0   565.0     2.0            1   
 
    HomePlanet_Europa  HomePlanet_Mars  CryoSleep_True  Cabin_B  Cabin_C  \
 0               True            False           False     True    False   
 1              False            False           False    False    False   
 2               True            False           False    False    False   
 3               True            False           False    False    False   
 4              False            False           False    False    False   
 
    Cabin_D  Cabin_E  Cabin_F  Cabin_G  Cabin_T  Dest

**Perform Train Test Split**

In [10]:
#your code here
from sklearn.model_selection import train_test_split

# Separamos features y target
y = spaceship_dum["Transported"].astype(int)
X = spaceship_dum.drop(columns=["Transported"]).astype(float)

# Split 80/20 estratificado
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.20,
    random_state=42,
    stratify=y
)

X_train.shape, X_test.shape, y_train.value_counts(normalize=True).round(3), y_test.value_counts(normalize=True).round(3)

((5284, 19),
 (1322, 19),
 Transported
 1    0.504
 0    0.496
 Name: proportion, dtype: float64,
 Transported
 1    0.504
 0    0.496
 Name: proportion, dtype: float64)

**Model Selection**

In this exercise we will be using **KNN** as our predictive model.

In [11]:
#your code here
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

# KNN clasificador con escalado (muy importante en KNN)
knn = make_pipeline(
    StandardScaler(with_mean=False),  # with_mean=False evita problemas si X es esparsa (por si acaso)
    KNeighborsClassifier(n_neighbors=5)
)

# Entrenamos
knn.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler(with_mean=False)),
                ('kneighborsclassifier', KNeighborsClassifier())])

- Evaluate your model's performance. Comment it

In [12]:
#your code here
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

y_pred = knn.predict(X_test)

acc = accuracy_score(y_test, y_pred)
cm  = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred, target_names=["Not Transported","Transported"])

print(f"Accuracy: {acc:.4f}\n")
print("Confusion matrix:\n", cm, "\n")
print("Classification report:\n", report)

Accuracy: 0.7602

Confusion matrix:
 [[490 166]
 [151 515]] 

Classification report:
                  precision    recall  f1-score   support

Not Transported       0.76      0.75      0.76       656
    Transported       0.76      0.77      0.76       666

       accuracy                           0.76      1322
      macro avg       0.76      0.76      0.76      1322
   weighted avg       0.76      0.76      0.76      1322

